In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import requests
import nltk
from nltk import tokenize
from fuzzywuzzy import fuzz
from bs4 import BeautifulSoup
from sklearn import preprocessing
from sklearn import model_selection



In [4]:
URL = r'https://batdongsan.vn/ban-nha/'

dict_html_parsing = [
                    {'body':''},\
                    {'div': 'landtech-container'},\
                    {'div':  "uk-container uk-container-center"},\
                    {'div': "block-custom realestate-post-detail-custom"},\
                    {'div': "uk-grid uk-grid-medium"},\
                    {'div': "uk-width-medium-3-5"},\
                    {'div': "project-global-object-block-003 information-custom"},\
                    {'div': "wrapper"},\
                    {'div': "uk-panel"}]

dict_html_house_title = dict_html_parsing.extend([{'h1': "uk-panel-title"},\
                                                {'span':''}])

dict_html_house_price = dict_html_parsing.extend([{'div': "item"},\
                                                {'div': "body"},\
                                                {'div': "meta"},\
                                                {'strong': "price"}])

dict_html_house_attribs = dict_html_parsing.extend([{'div': "item"},\
                                                {'div': "body"},\
                                                {'div': "param"},\
                                                {'ul', "uk-list"},\
                                                {'li': ''}])

header_features = ['Price', 'Title', 'Address', 'Area', 'Room', 'WC' , 'Des']


# Data Collection 


In [5]:
class Address:
    def __init__(self, city, district, ward, street):
        self._city = city
        self._district = district
        self._ward = ward
        self._street = street
    
    def display(self):
        print(self._street, self._ward, self._district, self._city)

In [12]:
class House:
    def __init__(self, title,  price ,address, area, room, wc, description):
        self._title = title
        self._price = price
        self._address = address
        self._area= area 
        self._room = room 
        self._wc = wc
        self._description = description
    
    def scrappingDataWeb(self, url):
        page = requests.get(url)
        soup_attrib = BeautifulSoup(page.content, "html.parser")
        soup_price = BeautifulSoup(page.content, "html.parser")
        soup_title = BeautifulSoup(page.content, "html.parser")
        
        # title parsing
        
        for item in dict_html_house_title:
            print(item.key())
            if item.value != '':
                soup_title = soup_title.find(item, class_ = item.value)
            else:
                soup_title = soup_title.find(item)
        self._title = soup_title.text
        
        # price parsing
        try:
            for item in dict_html_house_price:
                if item.value != '':
                    soup_price = soup_price.find(item, class_ = item.value)
                else:
                    soup_price = soup_price.find(item)
            self._price = soup_price.text
        except:
            print("error parsing price: ", url)
        
        ## parsing another attributes
        try:
            for item in dict_html_house_attribs:
                if item.value != '':
                    soup_attrib = soup_attrib.find(item, class_ = item.value)
                else:
                    soup_attrib = soup_attrib.find(item)
            attrib_root = soup_attrib.findall("i")
            for root in attrib_root:
                item = root.find("strong")
                if item.innerHTML =="Diện tích:":
                    self._area = item.text
                elif item.innerHTML =="Phòng ngủ:":
                    self._room == item.text
                elif item.innerHTML =="Địa chỉ:":
                    self._address = item.text
        except:
            print("error parsing attribs: ", url)
            
        ## chỉnh sửa phần này thành nhiều thành phần attib
        
    
    ## attributes matching for finding address base on The compare with voacabulary embedded database with Levhein distance
    def tranferDescription(self, embedded_database): 
        temp_address = Address('','','','')
        for city in embedded_database:
            for district in city._container:
                ## tokenize description scrapping from web to list vocab
                for token in tokenize(self._description):
                    # compare each vocab with each item address from db
                    score = fuzz(token, district)
                    if score >= 0.7:
                        Address._district = district
        return
    
    ##
    def tranferAddress(self):
        return
    def display(self):
        print(self._title,\
              self._price,\
              self._address,\
              self._area,\
              self._room,\
              self._wc,\
              self._description)

In [13]:
temp = House("","","","","","","")
temp.scrappingDataWeb("https://batdongsan.vn/ban-nha-pho-cu-loc-thanh-xuan-56m-4-tang-gia-545-ty-r268420")

TypeError: 'NoneType' object is not iterable

In [ ]:
class ContainerHouse:
    def __init__(self, _list = []):
        self._size = len(self._list)
    